<a href="https://colab.research.google.com/github/praveenjune17/English_Tamil_parallel_corpus/blob/master/Text_summarization/Abstractive_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install tensorflow-gpu==2.0.0-beta1
!pip install rouge==0.3.2
import tensorflow_datasets as tfds
import tensorflow as tf
tf.random.set_seed(100)
import time
import numpy as np

In [3]:
!tf_upgrade_v2 \
  --infile /usr/local/lib/python3.6/dist-packages/tensor2tensor/utils/beam_search.py \
  --outfile /usr/local/lib/python3.6/dist-packages/tensor2tensor/utils/beam_search.py
from tensor2tensor.utils.beam_search import beam_search
from rouge import Rouge

TensorFlow 2.0 Upgrade Script
-----------------------------
Converted 1 files
Detected 0 issues that require attention
--------------------------------------------------------------------------------


Make sure to read the detailed log 'report.txt'



In [0]:
examples, metadata = tfds.load('cnn_dailymail', with_info=True, as_supervised=True)
train, val, test = examples['train'], examples['validation'], examples['test']

In [0]:
# for doc, summ in examples['train']:
#   print('document summary',doc.numpy())
#   print('summary', summ.numpy())
#   break

In [0]:
# count=0
# for doc, summ in examples['train']:
#   count+=1
#print(count)   # Number of examples available 287113

In [0]:
train_examples = examples['train']
val_examples = examples['validation']
test_examples = examples['test']

In [0]:
# tokenizer_en = tfds.features.text.SubwordTextEncoder.build_from_corpus(
#     (doc.numpy() for doc, sum_r in train_examples), target_vocab_size=2**13)

In [0]:
tokenizer_en = tfds.features.text.SubwordTextEncoder.load_from_file('/content/vocab_file_summarization')

In [0]:
#tokenizer_en.save_to_file('/content/vocab_file_summarization')

In [11]:
sample_string = 'Transformer is awesome.'

tokenized_string = tokenizer_en.encode(sample_string)
print ('Tokenized string is {}'.format(tokenized_string))

original_string = tokenizer_en.decode(tokenized_string)
print ('The original string: {}'.format(original_string))

assert original_string == sample_string

Tokenized string is [8089, 1498, 8120, 201, 15, 2371, 3746, 277, 8051]
The original string: Transformer is awesome.


In [12]:
for ts in tokenized_string:
  print ('{} ----> {}'.format(ts, tokenizer_en.decode([ts])))

8089 ----> T
1498 ----> ran
8120 ----> s
201 ----> former 
15 ----> is 
2371 ----> aw
3746 ----> eso
277 ----> me
8051 ----> .


In [0]:
BUFFER_SIZE = 287113
BATCH_SIZE = 64

In [0]:
def encode(lang1, lang2):
  lang1 = [tokenizer_en.vocab_size] + tokenizer_en.encode(
      lang1.numpy()) + [tokenizer_en.vocab_size+1]

  lang2 = [tokenizer_en.vocab_size] + tokenizer_en.encode(
      lang2.numpy()) + [tokenizer_en.vocab_size+1]
  
  return lang1, lang2

In [0]:
doc_length = 512   #(200,75) worked,  , crashed at 230, 75 (275-305)
summ_length = 70
MAX_LENGTH = summ_length

In [0]:
# Set threshold for document and  summary length
def filter_max_length(x, y):
  return tf.logical_and(tf.size(x) <= doc_length,
                        tf.size(y) <= summ_length)

In [0]:
def tf_encode(doc, summary):
  return tf.py_function(encode, [doc, summary], [tf.int64, tf.int64])

In [18]:
train_dataset = train_examples.map(tf_encode)
train_dataset = train_dataset.filter(filter_max_length)
# cache the dataset to memory to get a speedup while reading from it.
#print('Number of records in train set after filter', sum(1 for _, _ in train_dataset)) # 
# 835 recs with doc_length = 180 & summ_length = 75
# 3225 recs with doc_length = 250 & summ_length = 75
# 2250 recs with doc_length = 225 & summ_length = 75
# 1350 recs with doc_length = 240 & summ_length = 50
# 1488 recs with doc_length = 220 & summ_length = 60
# 29176 recs with doc_length = 512 and summ_length = 70 (validation set = 1409)
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(BUFFER_SIZE, seed = 100).padded_batch(
    BATCH_SIZE, padded_shapes=([-1], [-1]))
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)


val_dataset = val_examples.map(tf_encode)
val_dataset = val_dataset.filter(filter_max_length)
#print('Number of records in train set after filter', sum(1 for _, _ in val_dataset))
# .padded_batch(
#     BATCH_SIZE, padded_shapes=([-1], [-1]))
val_dataset = val_dataset.padded_batch(
    BATCH_SIZE, padded_shapes=([-1], [-1]))
val_dataset = val_dataset.prefetch(tf.data.experimental.AUTOTUNE)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [67]:
doc_batch, summar_batch = next(iter(val_dataset))
doc_batch, summar_batch

(<tf.Tensor: id=161628, shape=(64, 509), dtype=int64, numpy=
 array([[8261,    1, 6082, ...,    0,    0,    0],
        [8261,  435, 5578, ...,    0,    0,    0],
        [8261,    5, 2838, ...,    0,    0,    0],
        ...,
        [8261,   75, 1343, ...,    0,    0,    0],
        [8261, 5182,  238, ...,    0,    0,    0],
        [8261,    5, 1035, ...,    0,    0,    0]])>,
 <tf.Tensor: id=161629, shape=(64, 70), dtype=int64, numpy=
 array([[8261,  423,    5, ...,    0,    0,    0],
        [8261,  435, 5578, ...,    0,    0,    0],
        [8261,    5, 2838, ...,    0,    0,    0],
        ...,
        [8261, 2475, 1064, ...,    0,    0,    0],
        [8261, 5182, 2038, ...,  228,  344, 8262],
        [8261,  463, 8114, ...,    0,    0,    0]])>)

In [0]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)
  
  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
  
  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    
  pos_encoding = angle_rads[np.newaxis, ...]
    
  return tf.cast(pos_encoding, dtype=tf.float32)
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  
  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask  # (seq_len, seq_len)
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead) 
  but it must be broadcastable for addition.
  
  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.
    
  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
  
  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)  

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    
    assert d_model % self.num_heads == 0
    
    self.depth = d_model // self.num_heads
    
    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)
    
    self.dense = tf.keras.layers.Dense(d_model)
        
  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])
    
  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]
    
    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)
    
    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
    
    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)
    
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention, 
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        
    return output, attention_weights
  
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    
  def call(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)
    
    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)
    
    return out2
  
  
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model, num_heads)
    self.mha2 = MultiHeadAttention(d_model, num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)
 
    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)
    
    
  def call(self, x, enc_output, training, 
           look_ahead_mask, padding_mask):
    # enc_output.shape == (batch_size, input_seq_len, d_model)

    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
    attn1 = self.dropout1(attn1, training=training)
    out1 = self.layernorm1(attn1 + x)
    
    attn2, attn_weights_block2 = self.mha2(
        enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
    attn2 = self.dropout2(attn2, training=training)
    out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)
    
    ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
    ffn_output = self.dropout3(ffn_output, training=training)
    out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)
    
    return out3, attn_weights_block1, attn_weights_block2
  
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, 
               rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    
    self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.pos_encoding = positional_encoding(input_vocab_size, self.d_model)
    
    
    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
  
    self.dropout = tf.keras.layers.Dropout(rate)
        
  def call(self, x, training, mask):

    seq_len = tf.shape(x)[1]
    
    # adding embedding and position encoding.
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)
    
    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)
    
    return x  # (batch_size, input_seq_len, d_model)

In [0]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, 
               rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    
    self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
    self.pos_encoding = positional_encoding(target_vocab_size, d_model)
    
    self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(rate)
    
  def call(self, x, enc_output, training, 
           look_ahead_mask, padding_mask):

    seq_len = tf.shape(x)[1]
    attention_weights = {}
    
    x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]
    
    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                             look_ahead_mask, padding_mask)
      
      attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
      attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    
    # x.shape == (batch_size, target_seq_len, d_model)
    return x, attention_weights

In [0]:
class Transformer(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, 
               target_vocab_size, rate=0.1):
    super(Transformer, self).__init__()

    self.encoder = Encoder(num_layers, d_model, num_heads, dff, 
                           input_vocab_size, rate)

    self.decoder = Decoder(num_layers, d_model, num_heads, dff, 
                           target_vocab_size, rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    
    self.generator_vec = tf.keras.layers.Dense(1, activation='sigmoid' )
    
  def call(self, inp, tar, training, enc_padding_mask, 
           look_ahead_mask, dec_padding_mask):

    enc_output = self.encoder(inp, training, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)
    
    # dec_output.shape == (batch_size, tar_seq_len, d_model)
    dec_output, attention_weights = self.decoder(
        tar, enc_output, training, look_ahead_mask, dec_padding_mask)
    
    final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)
    #print((attention_weights['decoder_layer{}_block2'.format(num_layers)]).shape)
    return final_output, attention_weights, dec_output

In [0]:
# # add a dense layer (dff, 1) with sigmoid activation 

# # take mean of all the attention heads
# updates = tf.reduce_mean(attention_weights['decoder_layer{}_block2'.format(num_layers)], axis=1) #(batch_size, tar_seq_len, inp_seq_len)
# updates = tf.transpose(updates, perm=[2 ,1, 0]) #(input_seq_len, tar_seq, batch)
# tar_seq_len = updates.shape[1]
# batch_size = updates.shape[-1]
# shape = tf.constant([target_vocab_size, tar_seq_len, batch_size]) #rename batch_size when training 
# indices = tf.transpose(encoder_input)
# copy_logits_ = tf.scatter_nd(indices, updates, shape)   # output shape (target_vocab_size, tar_seq_len, batch_size)
# copy_logits = tf.transpose(copy_logits_, perm=[2, 1, 0])
# logits = gen_logits + copy_logits

In [0]:
# transformer = Transformer(num_layers, d_model, num_heads, dff,
#                           input_vocab_size, target_vocab_size, dropout_rate)
# # tar_inp = tar[:, :-1]
# # tar_real = tar[:, 1:]

# # enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)
# # predictions, attention_weights, dec_output = transformer(inp, tar_inp, 
# #                                  True, 
# #                                  enc_padding_mask, 
# #                                  combined_mask, 
# #                                  dec_padding_mask,
# #                                  enable_copy=True)

In [0]:
# num_layers = 4  #number of transformer blocks
# d_model = 768   #the projected word vector dimension
# dff = 512       #
# num_heads = 12 

In [0]:
# # num_layers = 3  #number of transformer blocks
# # d_model = 768   #the projected word vector dimension
# # dff = 512       #
# # num_heads = 12   #the number of heads in the multi-headed attention unit
# Epoch 66 Batch 0 Loss 0.0015 Accuracy 0.9074
# Epoch 66 Loss 0.0107 Accuracy 0.8711
# Time taken for 1 epoch: 60.90722155570984 secs

In [0]:
# num_layers = 8
# d_model = 768
# dff = 2048
# num_heads = 8

# input_vocab_size = tokenizer_en.vocab_size + 2
# target_vocab_size = input_vocab_size
# dropout_rate = 0.1
num_layers = 3  #number of transformer blocks
d_model = 512   #the projected word vector dimension
dff = 512       #feed forward/filter size (in BERT its 4*d_model)
num_heads = 8   #the number of heads in the multi-headed attention unit

input_vocab_size = tokenizer_en.vocab_size + 2
target_vocab_size = input_vocab_size
dropout_rate = 0.0

In [0]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps
    
  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)
    
    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [0]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [0]:
def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  
  return tf.reduce_mean(loss_)

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')

In [0]:
transformer = Transformer(num_layers, d_model, num_heads, dff,
                          input_vocab_size, target_vocab_size, dropout_rate)

In [0]:
def create_masks(inp, tar):
  # Encoder padding mask
  enc_padding_mask = create_padding_mask(inp)
  
  # Used in the 2nd attention block in the decoder.
  # This padding mask is used to mask the encoder outputs.
  dec_padding_mask = create_padding_mask(inp)
  
  # Used in the 1st attention block in the decoder.
  # It is used to pad and mask future tokens in the input received by 
  # the decoder.
  look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
  dec_target_padding_mask = create_padding_mask(tar)
  combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
  
  return enc_padding_mask, combined_mask, dec_padding_mask

In [0]:
#checkpoint_path = "/content/drive/My Drive/Neural_machine_translation/Text_summarization_training"
checkpoint_path = "/content/temp"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print (ckpt_manager.latest_checkpoint, 'checkpoint restored!!')

In [0]:
EPOCHS = 100

In [0]:
# if enable_copy:
#       p_gen = self.generator_vec(dec_output) # (batch_size, tar_seq_len, 1)
#       vocab_dist_ = p_gen * final_output
#       # mean of all the attention heads
#       # updates shape (batch_size, tar_seq_len, inp_seq_len)
#       #print(tf.reduce_mean(attention_weights['decoder_layer{}_block2'.format(num_layers)], axis=1).shape)
#       updates = (1 - p_gen) * tf.reduce_mean(attention_weights['decoder_layer{}_block2'.format(num_layers)], axis=1) 
#       #print(tf.reduce_mean(attention_weights['decoder_layer{}_block2'.format(num_layers)], axis=1))
#       #print(updates.shape)
#       updates = tf.transpose(updates, perm=[2, 1, 0])
#       batch_size = updates.shape[-1]
#       # since the target input is shifted right so reducing the summary length
#       shape = tf.constant([target_vocab_size, summ_length-1, batch_size]) #rename batch_size when training 
#       #shape = tf.constant([BATCH_SIZE, summ_length-1, target_vocab_size]) #rename batch_size when training 
#       indices = tf.transpose(inp)#, perm=[1, 0])
#       indices = tf.cast(indices, dtype=tf.int32)
#       #indices = tf.expand_dims(indices, axis=1)
#       print('updates {}'.format(updates.shape))
#       print('indices {}'.format(indices.shape))
#       print('shape {}'.format(shape))
#       copy_logits_ = tf.scatter_nd(indices, updates, shape)   # output shape (target_vocab_size, tar_seq_len, batch_size)
#       #copy_logits = tf.transpose(copy_logits_, perm=[2, 1, 0])
#       final_output = p_gen + copy_logits
#       #final_output = tf.math.log(prob_output)
#       # (batch_size, tar_seq, inp_seq)

In [0]:
# class Generator(tf.keras.Model):
#   def __init__(self):
#     super(Generator, self).__init__()
#     self.generator_vec = tf.keras.layers.Dense(1, activation='sigmoid')
#     self.final_layer = tf.keras.layers.Dense(target_vocab_size)
#   def call(self, dec_output, final_output, attention_weights, encoder_input, training):  
#     p_gen = self.generator_vec(dec_output) # (batch_size, tar_seq_len, 1)
#     vocab_dist_ = p_gen * final_output
#     updates = (1 - p_gen) * tf.reduce_mean(attention_weights['decoder_layer{}_block2'.format(num_layers)], axis=1) 
#     updates = tf.transpose(updates, perm=[2, 0, 1])
#     #batch_size = updates.shape[-1]
#     # since the target input is shifted right so reducing the summary length
#     shape = tf.constant([target_vocab_size, BATCH_SIZE, summ_length-1]) #rename batch_size when training 
#     #shape = tf.constant([BATCH_SIZE, summ_length-1, target_vocab_size]) #rename batch_size when training 
#     #indices = tf.transpose(inp)#, perm=[1, 0])
#     indices = tf.cast(encoder_input, dtype=tf.int32)
#     #indices = tf.expand_dims(indices, axis=1)
#     #updates = tf.expand_dims(updates, axis=3)
#     #shape = tf.expand_dims(shape, axis=1)
#     indices = tf.reshape(indices[0], [509, -1])
#     print('updates {}'.format(updates.shape))
#     print('indices {}'.format(indices.shape))
#     print('shape {}'.format(shape))
#     copy_logits_ = tf.scatter_nd(indices, updates, shape)   # output shape (target_vocab_size, tar_seq_len, batch_size)
#     copy_logits = tf.transpose(copy_logits_, perm=[1, 2, 0])
#     combined_output = final_output + copy_logits
#     final_output = self.final_layer(combined_output)  
#     return final_output

In [142]:
tf.constant([BATCH_SIZE, 70-1, target_vocab_size]) 

<tf.Tensor: id=548181, shape=(3,), dtype=int32, numpy=array([  64,   69, 8263], dtype=int32)>

In [0]:
class Generator(tf.keras.Model):
  def __init__(self):
    super(Generator, self).__init__()
    self.generator_vec = tf.keras.layers.Dense(1, activation='sigmoid')
    self.final_layer = tf.keras.layers.Dense(target_vocab_size)
  def call(self, dec_output, final_output, attention_weights, encoder_input, inp_shape, tar_shape, batch, training):  
    #batch, summ_len = inp_shape
    p_gen = self.generator_vec(dec_output) # (batch_size, tar_seq_len, 1)
    vocab_dist_ = p_gen * final_output
    updates = (1 - p_gen) * tf.reduce_mean(attention_weights['decoder_layer{}_block2'.format(num_layers)], axis=1) 
    updates = tf.transpose(updates, perm=[0, 2, 1])
    tf.print(tar_shape-1)
    #shape = tf.Variable([batch, tar_shape-1, target_vocab_size]) 
    shape = [batch, tar_shape-1, target_vocab_size]
    i1, i2 = tf.meshgrid(tf.range(batch),
                     tf.range(inp_shape), indexing="ij")
    i1 = tf.tile(i1[:, :, tf.newaxis], [1, 1, tar_shape-1])
    i2 = tf.tile(i2[:, :, tf.newaxis], [1, 1, tar_shape-1])
    inp = tf.cast(encoder_input, dtype=tf.int32)
    indices = tf.tile(inp[:, :, tf.newaxis], [1, 1, tar_shape-1])
    indices = tf.stack([i1, i2, indices], axis=-1)
    copy_logits = tf.scatter_nd(indices, updates, shape)   
    combined_output = final_output + copy_logits
    final_output = self.final_layer(combined_output)  
    #tf.print(final_output.shape)
    return final_output

In [0]:
# class Generator(tf.keras.Model):
#   def __init__(self):
#     super(Generator, self).__init__()
#     self.generator_vec = tf.keras.layers.Dense(1, activation='sigmoid')
#     self.final_layer = tf.keras.layers.Dense(target_vocab_size)
#   def call(self, dec_output, final_output, attention_weights, encoder_input,training):  
#     p_gen = self.generator_vec(dec_output) # (batch_size, tar_seq_len, 1)
#     vocab_dist_ = p_gen * final_output
#     updates = (1 - p_gen) * tf.reduce_mean(attention_weights['decoder_layer{}_block2'.format(num_layers)], axis=1) 
#     updates = tf.transpose(updates, perm=[0, 2, 1])
#     shape = tf.constant([BATCH_SIZE, summ_length-1, target_vocab_size]) 
#     copy_logits = tf.scatter_nd(indices, updates, shape)   
#     combined_output = final_output + copy_logits
#     final_output = self.final_layer(combined_output)  
#     return final_output

In [0]:
# i1, i2 = tf.meshgrid(tf.range(64),
#                      tf.range(509), indexing="ij")
# i1 = tf.tile(i1[:, :, tf.newaxis], [1, 1, 68])
# i2 = tf.tile(i2[:, :, tf.newaxis], [1, 1, 68])
# inp = tf.cast(inp, dtype=tf.int32)
# indices = tf.tile(inp[:, :, tf.newaxis], [1, 1, 68])
# idx = tf.stack([i1, i2, indices], axis=-1)

In [0]:
generator = Generator()

# The @tf.function trace-compiles train_step into a TF graph for faster
# execution. The function specializes to the precise shape of the argument
# tensors. To avoid re-tracing due to the variable sequence lengths or variable
# batch sizes (the last batch is smaller), use input_signature to specify
# more generic shapes.

train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
    tf.TensorSpec(shape=(None), dtype=tf.int32),
    tf.TensorSpec(shape=(None), dtype=tf.int32),
    tf.TensorSpec(shape=(None), dtype=tf.int32),
]

@tf.function(input_signature=train_step_signature)
def train_step(inp, tar, inp_shape, tar_shape, batch):
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]
  
  enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)
  
  with tf.GradientTape() as tape:
    predictions, attention_weights, dec_output = transformer(inp, tar_inp, 
                                 True, 
                                 enc_padding_mask, 
                                 combined_mask, 
                                 dec_padding_mask)
    final_predictions = generator(dec_output, predictions, attention_weights, inp, inp_shape, tar_shape, batch, True)
    #tf.print(tar_real.shape, final_predictions.shape)
    loss = loss_function(tar_real, final_predictions)

  gradients = tape.gradient(loss, transformer.trainable_variables)    
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
  
  train_loss(loss)
  train_accuracy(tar_real, predictions)

In [158]:
for (batch, (inp, tar)) in enumerate(val_dataset):
  print(inp.shape,'  ', tar.shape)

(64, 509)    (64, 70)
(64, 507)    (64, 70)
(64, 510)    (64, 70)
(64, 511)    (64, 70)
(64, 509)    (64, 70)
(64, 505)    (64, 70)
(64, 511)    (64, 70)
(64, 512)    (64, 69)
(64, 509)    (64, 70)
(64, 504)    (64, 70)
(64, 503)    (64, 70)
(64, 505)    (64, 70)
(64, 504)    (64, 70)
(64, 509)    (64, 69)
(64, 501)    (64, 69)
(64, 496)    (64, 68)
(64, 512)    (64, 70)
(64, 510)    (64, 70)
(64, 511)    (64, 70)
(64, 510)    (64, 70)
(64, 512)    (64, 70)
(64, 512)    (64, 70)
(1, 263)    (1, 56)


In [0]:
# tar_inp = tar[:, :-1]
# tar_real = tar[:, 1:]
# enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)
# predictions, attention_weights, dec_output = transformer(inp, tar_inp, 
#                                  True, 
#                                  enc_padding_mask, 
#                                  combined_mask, 
#                                  dec_padding_mask)
# final_predictions = generator(dec_output, predictions, attention_weights, inp, True)

In [0]:
for epoch in range(EPOCHS):
  start = time.time()
  
  train_loss.reset_states()
  train_accuracy.reset_states()
  
  
  for (batch, (inp, tar)) in enumerate(val_dataset):
    if batch==0 and epoch ==0:
      print('Time taken for feeding the data to the model is {} seconds'.format(time.time()-start)) #takes 25 mins 
    train_step(inp, tar, inp.shape[1], tar.shape[1], inp.shape[0])
    
    if batch % 50 == 0:
      print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
          epoch + 1, batch, train_loss.result(), train_accuracy.result()))
      
  if (epoch + 1) % 1 == 0:
    ckpt_save_path = ckpt_manager.save()  
    print ('Saving checkpoint for epoch {} and batch at {} path is {}'.format(epoch+1,
                                                                     batch,
                                                                     ckpt_save_path))
    if train_loss.result() == 0:
      print('converged at epoch {} for model with parameters{}'.format(epoch+1, 
                                                     (num_layers, 
                                                     d_model, 
                                                     dff, 
                                                     num_heads)))
      break
    
  print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                                train_loss.result(), 
                                                train_accuracy.result()))

  print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Time taken for feeding the data to the model is 3.0572967529296875 seconds
69
Epoch 1 Batch 0 Loss 6.9656 Accuracy 0.0002
69
69
69
69
69
69
68
69
69
69
69
69
68
68
67
69
69
69
69
69
69
55
Saving checkpoint for epoch 1 and batch at 22 path is /content/temp/ckpt-1
Epoch 1 Loss 6.8790 Accuracy 0.0001
Time taken for 1 epoch: 151.39122486114502 secs

69
Epoch 2 Batch 0 Loss 6.5429 Accuracy 0.0000
69
69
69
69
69
69
68
69
69
69
69
69
68
68
67
69
69
69
69
69
69
55
Saving checkpoint for epoch 2 and batch at 22 path is /content/temp/ckpt-2
Epoch 2 Loss 6.4021 Accuracy 0.0000
Time taken for 1 epoch: 75.30403089523315 secs

69
Epoch 3 Batch 0 Loss 6.1357 Accuracy 0.0000
69
69
69
69
69
68
69


In [0]:
def evaluate(inp_sentence):
  start_token = [tokenizer_en.vocab_size]
  end_token = [tokenizer_en.vocab_size + 1]
  
  # inp sentence is portuguese, hence adding the start and end token
  inp_sentence = start_token + tokenizer_en.encode(inp_sentence) + end_token
  encoder_input = tf.expand_dims(inp_sentence, 0)
  
  # as the target is english, the first word to the transformer should be the
  # english start token.
  decoder_input = [tokenizer_en.vocab_size]
  output = tf.expand_dims(decoder_input, 0)
    
  for i in range(MAX_LENGTH):
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
        encoder_input, output)
  
    # predictions.shape == (batch_size, seq_len, vocab_size)
    predictions, attention_weights, _ = transformer(encoder_input, 
                                                 output,
                                                 False,
                                                 enc_padding_mask,
                                                 combined_mask,
                                                 dec_padding_mask)
    temp = predictions
    # select the last word from the seq_len dimension
    predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)

    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
    
    # return the result if the predicted_id is equal to the end token
    if predicted_id == tokenizer_en.vocab_size+1:
      return tf.squeeze(output, axis=0), attention_weights
    
    # concatentate the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0), attention_weights, temp, encoder_input

In [0]:
def summarize(sentence):
  result, attention_weights, logits, encoder_input = evaluate(sentence)
  
  predicted_sentence = tokenizer_en.decode([i for i in result 
                                            if i < tokenizer_en.vocab_size])  

  print('Input: {}'.format(sentence))
  print('Predicted translation: {}'.format(predicted_sentence))
  return (result, attention_weights, logits, encoder_input)

In [0]:
# generator_vec = tf.keras.layers.Dense(1, activation='sigmoid' )
# p_gen = generator_vec(gen_logits) # (batch_size, tar_seq_len, 1)
# #vocab_dist_ = p_gen * final_output
# # mean of all the attention heads
# # updates shape (batch_size, tar_seq_len, inp_seq_len)
# updates = (1 - p_gen) * tf.reduce_mean(attention_weights['decoder_layer{}_block2'.format(num_layers)], axis=1)
# updates = tf.transpose(updates, perm=[2, 1, 0])
# batch_size = updates.shape[-1]
# # since the target input is shifted right so reducing the summary length
# shape = tf.constant([target_vocab_size, summ_length-1, batch_size]) #rename batch_size when training 
# #shape = tf.constant([BATCH_SIZE, summ_length-1, target_vocab_size]) #rename batch_size when training 
# indices = tf.transpose(inp)#, perm=[1, 0])
# indices = tf.cast(indices, dtype=tf.int32)
# #indices = tf.expand_dims(indices, axis=1)
# print('updates {}'.format(updates.shape))
# print('indices {}'.format(indices.shape))
# print('shape {}'.format(shape))
# copy_logits_ = tf.scatter_nd(indices, updates, shape)   # output shape (target_vocab_size, tar_seq_len, batch_size)
# #copy_logits = tf.transpose(copy_logits_, perm=[2, 1, 0])
# final_output = p_gen + copy_logits

In [0]:
# Greedy search output
sample_document = '''Our pointer-generator network improved our ROUGE scores from our baseline; further analysis of the
summaries outputted by this model show that while we successfully reduced OOV mishandling, we
still suffer from factual inaccuracies. For instance, in the example below, it is clear that we no longer
run into out of vocabulary words; indeed, the model uses words like "yanukovych", "sergei", and
"oleg" in its summary, despite the fact that these are uncommon words that our baseline transformer
could not generate'''
result, attention_weights, gen_logits, encoder_input = summarize(sample_document)

In [0]:
# add a dense layer (dff, 1) with sigmoid activation 

# take mean of all the attention heads
updates = tf.reduce_mean(attention_weights['decoder_layer{}_block2'.format(num_layers)], axis=1) #(batch_size, tar_seq_len, inp_seq_len)
updates = tf.transpose(updates, perm=[2 ,1, 0]) #(input_seq_len, tar_seq, batch)
tar_seq_len = updates.shape[1]
batch_size = updates.shape[-1]
shape = tf.constant([target_vocab_size, tar_seq_len, batch_size]) #rename batch_size when training 
indices = tf.transpose(encoder_input)
copy_logits_ = tf.scatter_nd(indices, updates, shape)   # output shape (target_vocab_size, tar_seq_len, batch_size)
copy_logits = tf.transpose(copy_logits_, perm=[2, 1, 0])
logits = gen_logits + copy_logits

In [0]:
encoder_input

In [0]:
updates.shape

In [0]:
shape

In [0]:
indices.shape

In [0]:
# class Generator(tf.keras.Model):
#   def __init__(self, dff):
#     super(Generator, self).__init__()

# #     self.encoder = Encoder(num_layers, d_model, num_heads, dff, 
# #                            input_vocab_size, rate)

# #     self.decoder = Decoder(num_layers, d_model, num_heads, dff, 
# #                            target_vocab_size, rate)

#     self.generator_vec = tf.keras.layers.Dense(dff, activation='sigmoid')
    
#   def call(self, dec_output, final_output, attention_weights, encoder_input, #encoder_input :-first argument of transformer _call
#            training):  
#     p_gen = self.generator_vec(dec_output)
#     vocab_dist = tf.math.softmax(final_output, axis=-1)
#     vocab_dist_ = p_gen * vocab_dist
#     attn_dist_ = tf.math.softmax(attention_weights['decoder_layer{}_block2'.format(num_layers)], dim=-1)
#     attn_dist_ = (1 - p_gen) * attn_dist_
#     updates = tf.reduce_mean(attn_dist_, axis=1) #(batch_size, tar_seq_len, inp_seq_len)
#     updates = tf.transpose(updates, perm=[2 ,1, 0])
#     shape = tf.constant([target_vocab_size, summ_length, BATCH_SIZE]) #rename batch_size when training 
#     indices = tf.transpose(encoder_input)
#     copy_logits_ = tf.scatter_nd(indices, updates, shape)   # output shape (target_vocab_size, tar_seq_len, batch_size)
#     copy_logits = tf.transpose(copy_logits_, perm=[2, 1, 0])
#     logits = p_gen + copy_logits
    
#     return logits

In [0]:
# Note remove pad code when training (the ids would be padded already by the padd_batch step of tf.data)
#
def beam_search_train(inp_sentences, beam_size):
  
  start = [tokenizer_en.vocab_size] * len(inp_sentences)
  end = [tokenizer_en.vocab_size+1]
  pad_token = 0
  new = []
  inp_sentences = [tokenizer_en.encode(i) for i in inp_sentences]
  N = doc_length                                                         # set N as length of the sentence with max len
  inp_sentences = [[i]*beam_size for i  in inp_sentences]
  encoder_input = tf.reshape(tf.convert_to_tensor(inp_sentences), [-1, N]) # (batch_size * beam_size, doc_length)
  def transformer_query(output):

    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
          encoder_input, output)
    predictions, attention_weights = transformer(encoder_input, 
                                                 output,
                                                 True,
                                                 enc_padding_mask,
                                                 combined_mask,
                                                 dec_padding_mask,
                                                 True)

    # select the last sequence
    return (predictions[:,-1:,:]) # (batch_size, 1, target_vocab_size)
  return beam_search(transformer_query, start, beam_size, 120, target_vocab_size, 0.6, stop_early=True, eos_id=[end])

In [0]:
# Note remove pad code when training (the ids would be padded already by the padd_batch step of tf.data)
def beam_search_eval(inp_sentences, beam_size):
  
  start = [tokenizer_en.vocab_size] * len(inp_sentences)
  end = [tokenizer_en.vocab_size+1]
  pad_token = 0
  new = []
  inp_sentences = [tokenizer_en.encode(i) for i in inp_sentences]
  N = len(max(inp_sentences, key=len))  # set N as length of the sentence with max len
  # Pad zero
  inp_sentences = [[i + [pad_token] * (N - len(i))]*beam_size if len(i) < N else [i]*beam_size for i  in inp_sentences]
  encoder_input = tf.reshape(tf.convert_to_tensor(inp_sentences), [-1, N])
  
  def transformer_query(output):
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
          encoder_input, output)
    predictions, attention_weights = transformer(encoder_input, 
                                                 output,
                                                 False,
                                                 enc_padding_mask,
                                                 combined_mask,
                                                 dec_padding_mask,
                                                 True)

    # select the last sequence
    return (predictions[:,-1:,:])  # (batch_size, 1, target_vocab_size)
  return beam_search(transformer_query, start, beam_size, summ_length, target_vocab_size, 0.6, stop_early=True, eos_id=[end])

In [0]:
inp_sentences = [sample_document]
beam_size = 7
start_time = time.time()
# translated_output_temp[0] =>  (batch, beam_size, summ_length+1)
translated_output_temp = beam_search_eval(inp_sentences, beam_size)

for true_summary, top_sentence_ids in zip(inp_sentences, translated_output_temp[0][:,0,:]):
  print('Original summary: {}'.format(true_summary))
  print('Predicted summary: {}'.format(tokenizer_en.decode([j for j in top_sentence_ids if j < tokenizer_en.vocab_size])))
print('time to process {}'.format(time.time()-start_time))

In [0]:
# # Logits needs to be returned from transformer query function
# def beam_search_eval(inp_sentence):
#   beam_size = 5
#   start_token = [tokenizer_en.vocab_size]
#   end_token = [tokenizer_en.vocab_size + 1]
#   inp_sentence = start_token + tokenizer_en.encode(inp_sentence) + end_token
#   encoder_input = tf.expand_dims(inp_sentence, 0)
#   encoder_input = tf.concat([encoder_input]*beam_size, axis=0)
#   start = tokenizer_ta.vocab_size
#   end = tokenizer_ta.vocab_size+1
#   def transformer_query(output):

#     enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
#           encoder_input, output)
#     predictions, attention_weights = transformer(encoder_input, 
#                                                  output,
#                                                  False,
#                                                  enc_padding_mask,
#                                                  combined_mask,
#                                                  dec_padding_mask)


#     return (predictions[:,-1:,:])
#   return beam_search(transformer_query, [start], beam_size, 120, target_vocab_size, 0.6, stop_early=False, eos_id=[tokenizer_ta.vocab_size+1])

In [0]:
# Add BERT weights : -computational issues
#

In [0]:
# histogram on number on number of total token length (doc+summary) vs training examples
# Neat script with the same pipeline as Tf-toy train
# add label smoothing
# copy mechanism 
